In [1]:
from Core_Game_Parts import *
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from collections import deque
import matplotlib.pyplot as plt
import random
import time
import os

c:\Users\Yash\OneDrive\Desktop\Neural Networks Car Project\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.13.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def collect_expert_data(filename="pretrain_data.npy"):
    """
    Run the simulation manually and save state-action pairs.
    """
    # Force a display window to open for manual play
    os.environ["SDL_VIDEODRIVER"] = "windows"
    
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    track_surface = pygame.image.load(TRACK_IMAGE_PATH).convert()
    car = Car(CAR_IMAGE_PATH, DEFAULT_START_X, DEFAULT_START_Y, angle=DEFAULT_START_ANGLE)
    
    driving_data = []
    running = True
    print("Starting data collection. Drive 3-5 clean laps. Press ESC or close window to finish.")

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
                running = False
        
        keys = pygame.key.get_pressed()
        action = None
        # Map keys to the agent's actions (0: Left, 1: Right, 2: Brake)
        if keys[pygame.K_LEFT] or keys[pygame.K_a]:  action = 0
        elif keys[pygame.K_RIGHT] or keys[pygame.K_d]: action = 1
        elif keys[pygame.K_DOWN] or keys[pygame.K_s]:  action = 2

        distances, _ = ray_casting(car, track_surface)
        normalized_speed = car.speed / MAX_SPEED
        state = np.array(distances + [normalized_speed])
        
        if action is not None:
            driving_data.append([state, action])

        # Standard manual driving physics
        if keys[pygame.K_UP] or keys[pygame.K_w]: car.speed += ACCELERATION
        if car.speed > 0:
            speed_factor = car.speed / MAX_SPEED
            turn = MAX_TURN_ANGLE - (speed_factor) * (MAX_TURN_ANGLE - MIN_TURN_ANGLE)
            if keys[pygame.K_LEFT] or keys[pygame.K_a]: car.angle += turn
            if keys[pygame.K_RIGHT] or keys[pygame.K_d]: car.angle -= turn
        if keys[pygame.K_DOWN] or keys[pygame.K_s]: car.speed -= BRAKE_FORCE
        car.speed -= FRICTION
        car.speed = max(0, min(car.speed, MAX_SPEED))
        car.move()

        screen.blit(track_surface, (0, 0)); car.draw(screen); pygame.display.update(); clock.tick(60)

    pygame.quit()
    
    if driving_data:
        print(f"Saving {len(driving_data)} data points to {filename}...")
        driving_data_array = np.array(driving_data, dtype=object)
        np.save(filename, driving_data_array, allow_pickle=True)
        print("Save complete!")


In [3]:
def model():
    """Builds the neural network model."""
    net = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax') # Softmax is better for imitation learning
    ])
    net.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return net

def pretrain_agent(data_path="pretrain_data.npy", weights_path="pretrained.weights.h5"):
    """
    Trains a model on the expert data using supervised learning.
    """
    print("Loading expert data...")
    expert_data = np.load(data_path, allow_pickle=True)
    
    states = np.array([item[0] for item in expert_data])
    actions = np.array([item[1] for item in expert_data])
    
    # Convert actions to one-hot encoding (e.g., 0 -> [1,0,0], 1 -> [0,1,0])
    actions_one_hot = to_categorical(actions, num_classes=3)
    
    print(f"Data loaded. Training on {len(states)} samples...")
    ai_model = model()
    
    ai_model.fit(states, actions_one_hot, epochs=15, batch_size=64, validation_split=0.1, shuffle=True)
    
    print(f"Pre-training complete. Saving weights to {weights_path}...")
    ai_model.save_weights(weights_path)
    print("Weights saved!")



In [4]:
def train_imitation_model(data_path="pretrain_data.npy", weights_path="final_imitation.weights.h5"):
    print("Loading expert data...")
    expert_data = np.load(data_path, allow_pickle=True)
    
    states = np.array([item[0] for item in expert_data])
    actions = np.array([item[1] for item in expert_data])
    actions_one_hot = to_categorical(actions, num_classes=3)
    
    print(f"Data loaded. Training on {len(states)} samples...")
    
    # Define the model for classification
    imitation_model = Sequential([
        Dense(64, activation='relu', input_shape=(4,)),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax') # Softmax for predicting the probability of each action
    ])
    imitation_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model to imitate
    imitation_model.fit(states, actions_one_hot, epochs=25, batch_size=64, validation_split=0.1, shuffle=True)
    
    print(f"Training complete. Saving final weights to {weights_path}...")
    imitation_model.save_weights(weights_path)
    print("Final weights saved!")


In [6]:
if __name__=="__main__":
    #collect_expert_data()  #for collecting the data
    pretrain_agent()       #for pre training
    train_imitation_model() #for Behavioral Training

Loading expert data...
Data loaded. Training on 2790 samples...


c:\Users\Yash\OneDrive\Desktop\Neural Networks Car Project\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7551 - loss: 1.0004 - val_accuracy: 0.6380 - val_loss: 0.7810
Epoch 2/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7487 - loss: 0.4412 - val_accuracy: 0.7276 - val_loss: 0.4984
Epoch 3/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7658 - loss: 0.4675 - val_accuracy: 0.6487 - val_loss: 0.9728
Epoch 4/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7662 - loss: 0.4289 - val_accuracy: 0.7348 - val_loss: 0.5257
Epoch 5/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7702 - loss: 0.4637 - val_accuracy: 0.7419 - val_loss: 0.4867
Epoch 6/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7714 - loss: 0.4057 - val_accuracy: 0.7312 - val_loss: 0.4674
Epoch 7/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7658 - loss: 0.4140 - val_accuracy: 0.7204 - val_loss: 0.4732
Epoch 8/15
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7698 - loss: 0.4642 - val_accuracy: 0.7348 - val_loss: